In [1]:
import pandas as pd
from viruscope import readfa
import os
import os.path as op
import glob
from collections import Counter

Note: to build an hmm database out of hmm profiles, concatenate the profiles into one file and then run ```hmmpress```

Downloaded all files found here: http://vogdb.org/download

In [2]:
wd = "/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/vog201/"

In [3]:
os.chdir(wd)

In [21]:
!wget http://fileshare.csb.univie.ac.at/vog/vog99/vog.raw_algs.tar.gz
!wget http://fileshare.csb.univie.ac.at/vog/vog99/vog.hmm.tar.gz
!wget http://fileshare.csb.univie.ac.at/vog/vog99/vog.faa.tar.gz    

--2020-05-19 13:54:31--  http://fileshare.csb.univie.ac.at/vog/vog99/vog.raw_algs.tar.gz
Resolving fileshare.csb.univie.ac.at (fileshare.csb.univie.ac.at)... 131.130.65.128
Connecting to fileshare.csb.univie.ac.at (fileshare.csb.univie.ac.at)|131.130.65.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37606872 (36M) [application/x-gzip]
Saving to: ‘vog.raw_algs.tar.gz’

100%[======================================>] 37,606,872  3.87MB/s   in 19s    

2020-05-19 13:54:50 (1.85 MB/s) - ‘vog.raw_algs.tar.gz’ saved [37606872/37606872]

--2020-05-19 13:54:51--  http://fileshare.csb.univie.ac.at/vog/vog99/vog.hmm.tar.gz
Resolving fileshare.csb.univie.ac.at (fileshare.csb.univie.ac.at)... 131.130.65.128
Connecting to fileshare.csb.univie.ac.at (fileshare.csb.univie.ac.at)|131.130.65.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 336759552 (321M) [application/x-gzip]
Saving to: ‘vog.hmm.tar.gz’

100%[=====================================

In [15]:
!mv ./*/*/*/* ./

```
tar -xzf vog.hmm.tar.gz
mkdir hmm
mv *.hmm hmm
tar -xzf vog.faa.tar.gz
mkdir indiv_vogs
mv *.faa indiv_vogs
cd hmm
cat *.hmm > vog_hmm
hmmpress vog_hmm
```

In [4]:
vogs = glob.glob(op.join(wd, "faa", "*.faa"))

In [5]:
len(vogs)

25743

In [6]:
test = vogs[0]
test

'/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/vog201/faa/VOG00001.faa'

In [7]:
rdf = pd.read_csv(op.join(wd, 'vog.annotations.tsv'), sep="\t")

In [9]:
Counter(rdf['ConsensusFunctionalDescription']).most_common()[:10]

[('REFSEQ hypothetical protein', 21419),
 ('REFSEQ structural protein', 68),
 ('REFSEQ minor tail protein', 42),
 ('REFSEQ membrane protein', 35),
 ('REFSEQ HNH endonuclease', 26),
 ('REFSEQ holin', 25),
 ('REFSEQ tail assembly chaperone', 23),
 ('REFSEQ putative tail fiber protein', 23),
 ('REFSEQ tail fiber protein', 21),
 ('REFSEQ putative virion structural protein', 19)]

In [10]:
with open(op.join(wd, "vog_proteins_nr.fasta"), "w") as oh:
    for v in vogs:
        seqs = []

        vog = op.basename(v).split(".")[0]
        fcat = rdf[rdf['#GroupName'] == vog]['FunctionalCategory'].values[0]
        fdesc = rdf[rdf['#GroupName'] == vog]['ConsensusFunctionalDescription'].values[0]

        for name, seq in readfa(open(v)):
            if seq not in seqs:
                print(">{name}__{fcat}__{vog}:{fdesc}".format(name=name.replace(" ","_"), vog=vog, fcat=fcat, fdesc=fdesc.replace(" ","_")), file=oh)
                for i in range(0, len(seq), 60):
                    print(seq[i:i+60], file=oh)

                seqs.append(seq)

In [11]:
!head {op.join(wd, "vog_proteins_nr.fasta")} -n200

>2547425.YP_009873962.1_DNA-binding_transcriptional_regulator_[Tenacibaculum_phage_PTm1]__Xu__VOG00001:sp|P03041|RPC1_BPP22_Transcriptional_activator_protein_C1
MNRIKEVLEGKGIKQIWLAEQLGKSYNMVHSYVQNKRQPSLEDLYKIAKILNVEVNELLV
SIKELN
>37105.YP_007003000.1_putative_Cro-like_repressor_[Lactobacillus_phage_JCL1032]__Xu__VOG00001:sp|P03041|RPC1_BPP22_Transcriptional_activator_protein_C1
MTVIKKGAGHMLKEEIDKRGLRYAFVAEKAGVAPQTLSKYLHDRIPINVNNLYKICHVVD
VDYKKFLK
>1445809.YP_009194019.1_helix-turn-helix_transcriptional_regulator_[Bacillus_phage_vB_BtS_BMBtp3]__Xu__VOG00001:sp|P03041|RPC1_BPP22_Transcriptional_activator_protein_C1
MTKMIKLNTERARELRENHGYTHGYVAKRLNCHKSAYSHIERGFRQPSIDKLGKLSELYG
VPTDELLIKSTL
>2301601.YP_009812740.1_hypothetical_protein_[Corynebacterium_phage_SamW]__Xu__VOG00001:sp|P03041|RPC1_BPP22_Transcriptional_activator_protein_C1
MTNAIRVRTDRMKNLTEIHGLNESETARRIGCSRQTYRRAIDGENVSAGFVAGACLSFGV
PFDALFHTVRVEAETPAA
>1582152.YP_009221774.1_helix-turn-helix_transcriptional_regulator_[Clostridium

In [12]:
!grep -c ">" {op.join(wd, "vog_proteins_nr.fasta")}

254305


In [13]:
!makeblastdb -in {op.join(wd, "vog_proteins_nr.fasta")} -dbtype prot 



Building a new DB, current time: 10/02/2020 12:34:10
New DB name:   /mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/vog201/vog_proteins_nr.fasta
New DB title:  /mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/vog201/vog_proteins_nr.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 254305 sequences in 8.11439 seconds.


In [6]:
!makeblastdb --help

USAGE
  makeblastdb [-h] [-help] [-in input_file] [-input_type type]
    -dbtype molecule_type [-title database_title] [-parse_seqids]
    [-hash_index] [-mask_data mask_data_files] [-mask_id mask_algo_ids]
    [-mask_desc mask_algo_descriptions] [-gi_mask]
    [-gi_mask_name gi_based_mask_names] [-out database_name]
    [-max_file_sz number_of_bytes] [-logfile File_Name] [-taxid TaxID]
    [-taxid_map TaxIDMapFile] [-version]

DESCRIPTION
   Application to create BLAST databases, version 2.2.31+

Use '-help' to print detailed descriptions of command line arguments

Error: Unknown argument: "-help"
